In [1]:

from app.log import logging_config
from dotenv import load_dotenv
import logging
from app.util.message import starting_message
from app import Polygon
from app import Sentiment_predict
import pandas as pd
from app.api import Twitter 
from os import environ


c:\Users\andre\Anaconda3\envs\DS4A_PF\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Polygon

In [ ]:
##Load and upgrade initial df
polygon_prev = pd.read_csv("polygon_df_01_07_2022.csv")
#Polygon credentials
load_dotenv()
print(environ.get("POLYGON_KEY"))
logging_config.init_logging()
__LOG = logging.getLogger(__name__)
__LOG.info('...... Initialization Completed  ......')
__LOG.info(starting_message())
#Initialize Polygon
polygon = Polygon(environ.get("POLYGON_KEY"))
#Download News
status, polygon_data = polygon.get_news('')
if status:
    polygon_news_df = pd.DataFrame.from_dict(polygon_data['results'])
polygon_news_df.drop(['publisher', 'keywords'], axis = 1, inplace = True)
polygon_news_df['tickers'] = [' '.join(map(str, l)) for l in polygon_news_df['tickers']]
#predict Sentiment
sp = Sentiment_predict()
polygon_news_df['title_finBERT'] = sp.sentiment_finBERT(polygon_news_df, 'title')
polygon_news_df['title_financialBERT'] = sp.sentiment_financialBERT(polygon_news_df, 'title')
polygon_news_df['description_finBERT'] = sp.sentiment_finBERT(polygon_news_df, 'description')
polygon_news_df['description_financialBERT'] = sp.sentiment_financialBERT(polygon_news_df, 'description')
#Upload df.
polygon_news_df = pd.concat([polygon_news_df, polygon_prev], ignore_index = True).drop_duplicates().reset_index(drop = True)
polygon_news_df.to_csv("polygon_df_01_07_2022.csv", index = False, encoding='utf-8')



PWRjSfmBc9xYnPZMQhA9ukDOuOIId90D


Token indices sequence length is longer than the specified maximum sequence length for this model (774 > 512). Running this sequence through the model will result in indexing errors


in row 501 the text is divided into two parts
in row 752 the text is divided into two parts
in row 978 the text is too long, no sentiment predicted


Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


in row 501 the text is divided into two parts
in row 752 the text is divided into two parts
in row 978 the text is too long, no sentiment predicted


In [ ]:
#Polygon plots Data
df_poly = polygon_news_df[['id','published_utc', 'title','description', 'title_finBERT', 'description_finBERT', 'title_financialBERT', 'description_financialBERT', 'article_url']]
df_poly['date'] = df_poly['published_utc'].str[0:10]
df_poly["total_news_day"] = df_poly.groupby('date')['id'].transform(len)
df_poly = df_poly.groupby(['date','title_finBERT']).size().reset_index()
df_poly.rename({0:'count'}, axis = 1, inplace = True)
df_poly

C:\Users\andre\AppData\Local\Temp\ipykernel_122108\4049042377.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_poly['date'] = df_poly['published_utc'].str[0:10]
C:\Users\andre\AppData\Local\Temp\ipykernel_122108\4049042377.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_poly["total_news_day"] = df_poly.groupby('date')['id'].transform(len)


,date,title_finBERT,count
0,2022-06-30,negative,85
1,2022-06-30,neutral,303
2,2022-06-30,positive,77
3,2022-07-01,negative,104
4,2022-07-01,neutral,351
5,2022-07-01,positive,89


## Tweets

In [ ]:
twt = Twitter()
df_twt = twt.get_tweets_df(ticker = 'BTC', query = 'bitcoin OR BTC AND crypto', limit = 100, popular =True)
df_twt['tweet_finBERT'] = sp.sentiment_finBERT(df_twt, 'full_text')
df_twt['tweet_financialBERT'] = sp.sentiment_financialBERT(df_twt, 'full_text')
df_twt.dtypes

id                     object
created_at             object
full_text              object
truncated                bool
retweets                int64
user_name              object
user_verified            bool
user_followers          int64
user_image_url         object
ticker                 object
tweet_finBERT          object
tweet_financialBERT    object
dtype: object

In [ ]:
month_number = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May': '05','Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

df_twt['day'] = df_twt['created_at'].str[7:10]
df_twt['month'] = df_twt['created_at'].str[4:7].map(month_number)
df_twt['year'] = df_twt['created_at'].str[-4:]

df_twt['date'] = df_twt['year'] + '-' + df_twt['month'] + '-' + df_twt['day'] 
#Twitter plots Data
df_twt["total_news_day"] = df_twt.groupby('date')['id'].transform(len)
twt_df = df_twt.groupby(['date', 'tweet_finBERT']).size().reset_index()
twt_df.rename({0:'count'}, axis = 1, inplace = True)
twt_df

,date,tweet_finBERT,count
0,2022-06- 22,neutral,14
1,2022-06- 22,positive,3
2,2022-06- 23,negative,1
3,2022-06- 23,neutral,4
4,2022-06- 23,positive,1
5,2022-06- 24,neutral,2
6,2022-06- 25,neutral,1
7,2022-06- 27,negative,1
8,2022-06- 27,neutral,1
9,2022-06- 28,neutral,2


## Dash plots

In [ ]:
import plotly.express as px
def sent_pie(df, count, col_name, plot_title):
    fig = px.pie(df, values = count, names = col_name, template='plotly_dark', title = plot_title)
    fig.update_layout(
    legend_traceorder="reversed",
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
        #family="Courier New, monospace",
        size=25)
        },
    )
    
    return fig
def sent_bar(df, date, count, col_name, plot_title):
    fig = px.bar(df, x=date, y=count, color=col_name, template='plotly_dark', category_orders= {col_name: ["neutral", "negative", "positive"]}, title= plot_title)
    fig.update_layout(
        legend_traceorder="reversed",
        legend_title = '',
        title={
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
            #family="Courier New, monospace",
            size=25)
            },
        )
    return fig

In [ ]:
sent_pie(twt_df, 'count', 'tweet_finBERT', "Tweets by sentiment").show()

In [ ]:
sent_pie(df_poly, 'count', 'title_finBERT', "News by sentiment").show()

In [ ]:
sent_bar(twt_df, 'date', 'count', 'tweet_finBERT', "Daily Tweets by Sentiment").show()


In [ ]:
sent_bar(df_poly, 'date', 'count', 'title_finBERT', "Daily News by Sentiment").show()